# GPS

## How does it work: Triangulation

<img style="width:40%" src="files/gps.jpg">
https://www.researchgate.net/publication/316658355_GPSGIS

## Lets visualize Triangulation!

In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import scipy.constants as constants

np.random.seed(983474)

### Time delay is a distance (speed of light) = A sphere around each satellite

In [ ]:
theta = np.linspace(0, 2 * np.pi, 100)
phi = np.linspace(0, np.pi, 100)

def sphere(p0, r):
    x = p0[0] + r*np.outer(np.cos(theta), np.sin(phi))
    y = p0[1] + r*np.outer(np.sin(theta), np.sin(phi))
    z = p0[2] + r*np.outer(np.ones(np.size(theta)), np.cos(phi))
    return x,y,z

### The setting

In [ ]:
satellites = [np.random.randn(3,1)*10 for i in range(4)] #4 satellites with x,y,z coordinates

our_position = np.ones((3,1)) #we are at this location [1,1,1]

ranges = [np.linalg.norm(sat - our_position) for sat in satellites]

### The plot

In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')

ax.plot(our_position[0], our_position[1], our_position[2], 'xg')

for sat in satellites:
    ax.plot(sat[0], sat[1], sat[2], '.r')
    ax.plot([sat[0,0], our_position[0,0]], [sat[1,0], our_position[1,0]], [sat[2,0], our_position[2,0]], '-k', alpha=0.3)

In [ ]:
for sat, r in zip(satellites, ranges):
    x, y, z = sphere(sat, r)
    ax.plot_surface(x, y, z,  rstride=4, cstride=4, color='b', linewidth=0, alpha=0.1)

# General relativity

One fact: Time slows down in more curved field

This means:
- Time slows down when you are affected by more gravity (close to the Earth)
- Time slows down when you go faster (Traveling around the Earth)

So, with respect to us, a satellite:
- Has a faster clock due to being higher up at ~20 200 km altitude
- Has a slower clock due to going around the Earth at ~4 km/s

## Accelerating frame time dialation

- $t = $ Time for us on Earth
- $t' = $ Time for the satellite
- $v = $ Satellite velocity

$$ \frac{t}{t'} = \frac{1}{\sqrt{1 - \frac{v^2}{c^2}}} $$

In [ ]:
v = 4e3 #m/s
t_frame_fraction = 1/np.sqrt(1 - v**2/constants.c**2)

print(f'Our time is {t_frame_fraction} of the Satellite time (i.e. faster)')

## Gravitational time dialation

- $t = $ Time unaffected by gravity
- $t' = $ Time affected by gravity
- $r = $ Observer radius from center of Earth
- $r_e = $ Radius of the Earth
- $r_{\mathrm{sat}} = $ Distance from center of earth to satellite
- $r_s = \frac{2GM}{c^2} $ Schwarzschild radius of the Earth (even though we are not a black hole, its fine)

$$ \frac{t'}{t} = \sqrt{1 - \frac{r_s}{r}} $$

Then time dialation difference between us and a satellite is

$$ \frac{t'_{\mathrm{sat}}}{\sqrt{1 - \frac{r_s}{r_{\mathrm{sat}}}}} = \frac{t'_{\mathrm{earth}}}{\sqrt{1 - \frac{r_s}{r_e}}} $$
$$ \frac{t'_{\mathrm{earth}}}{t'_{\mathrm{sat}}} = \frac{\sqrt{1 - \frac{r_s}{r_e}}}{\sqrt{1 - \frac{r_s}{r_{\mathrm{sat}}}}} $$

In [ ]:
M_earth = 5.97e24 # kg
r_e = 6.37e6 # m
r_s = 2*constants.G*M_earth/constants.c**2 # m

r_sat = 20.2e6 + r_e

t_field_fraction = np.sqrt(1 - r_s/r_e)/np.sqrt(1 - r_s/r_sat)
print(f'Our time is {t_field_fraction} of the Satellite time (i.e. slower)')

# Effect on GPS

## Time of a signal

How do we know the travel time? **CLOCKS**
- A clock on the satellite
- A clock on Earth

And we compare

## Uncorrected clocks

### Error per day

In [ ]:
dt = 24*60*60.0 #s 

t_err_frame = dt - dt*t_frame_fraction
print(f'Clock on GPS approx {t_err_frame*1e6:.0f} microseconds error per day due to Speed')

t_err_field = dt - dt*t_field_fraction
print(f'Clock on GPS approx {t_err_field*1e6:.0f} microseconds error per day due to Gravity field')

### Effects on distance calculation

In [ ]:
time_dialation = t_field_fraction*t_frame_fraction

time = np.linspace(0, 24, num=1000) # hours
time_s = time*60*60
time_error = time_s - time_dialation*time_s

fig, ax = plt.subplots(1,1)
ax.plot(time, time_error*1e6)
ax.set_xlabel('Time [h]')
ax.set_ylabel('Timing error [microseconds]')

In [ ]:
fig, ax = plt.subplots(1,1)
ax.plot(time, time_error*constants.c*1e-3)
ax.set_xlabel('Time [h]')
ax.set_ylabel('Distance error [km]')